### BELOW CODE IS TO WORK WITH SPARK IN COLAB, IGNORE IF NOT USING COLAB

In [1]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,374 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,174 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/mai

In [2]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
os.environ['PYSPARK_PYTHON']=sys.executable
os.environ['PYSPARK_DRIVER_PYTHON']=sys.executable

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('week6').getOrCreate()
spark

### 1)Demonstrate how to load a dataset suitable for prediction into a PySpark DataFrame and Display basic statistics and information about the dataset.

In [4]:
path='/content/covtype.data'
df = spark.read.csv(path,inferSchema=True,header=True)
colnames = ["Elevation","Aspect","Slope","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology",\
           "Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_noon","Hillshade_3pm",\
           "Horizontal_Distance_To_Fire_Points"] + \
           [f"Wilderness_Area_{i}" for i in range(4)] + [f"Soil_Type_{i}" for i in range(40)] + ["Cover_Type"]
df = df.toDF(*colnames)
df.show(5)
df.printSchema()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness

In [5]:
df.summary().show()

+-------+------------------+------------------+-----------------+--------------------------------+------------------------------+-------------------------------+------------------+------------------+------------------+----------------------------------+-------------------+-------------------+------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-----

### 2)Implement a PySpark script to handle missing values and categorical features in thedataset.

removing null values using imputer

In [7]:
from pyspark.ml.feature import Imputer
input_Cols=df.columns
output_Cols=[col+'_imputed' for col in inputCols]
removeNull=Imputer(inputCols=input_Cols, outputCols=output_Cols)
imputedDF=removeNull.fit(df).transform(df)

combining all independent variables into one feature vector


In [11]:
from pyspark.ml.feature import VectorAssembler
input_cols=df.columns[:-1]
assembler=VectorAssembler(inputCols=input_cols,outputCol='feature vector')
df=assembler.transform(df)

In [13]:
df.select(['feature vector','Cover_Type']).show(1)

+--------------------+----------+
|      feature vector|Cover_Type|
+--------------------+----------+
|(54,[0,1,2,3,4,5,...|         5|
+--------------------+----------+
only showing top 1 row



3) Develop a PySpark script that trains a decision tree model on the training dataset.


In [14]:
train,test=df.randomSplit([0.8,0.2])

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier
decisionTree=DecisionTreeClassifier(labelCol="Cover_Type",featuresCol='feature vector')
tree=decisionTree.fit(train)

In [16]:
predictions=tree.transform(test)
predictions.show(5)

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------+--------------------+--------------------+--------------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|

4)Implement code to evaluate the decision tree model using metrics such as accuracy,
precision, and recall.


In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
accuracyCalc=MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='Cover_Type',metricName='accuracy')
accuracy=accuracyCalc.evaluate(predictions)
print(f"accuracy={accuracy}")

accuracy=0.6995967741935484


In [21]:
precisionCalc=MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='Cover_Type',metricName='weightedPrecision')
precision=precisionCalc.evaluate(predictions)
print(f"precision={precision}")

precision=0.6788392509572623


In [20]:
recallCalc = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="prediction", metricName="weightedRecall")
recall = recallCalc.evaluate(predictions)
print(f"Recall: {recall}")

Recall: 0.6995967741935484
